In [1]:
using LinearAlgebra

# N spins
N = 12
Dim = 2^N
# couplings
J_X = 1
J_Y = 0


J_Z = 1
B = 0
#decay
alpha = 10

#function for the distance 1/(r_ij ^alpha)
function r_distance(index_i, index_j, alpha)
    dist = abs(index_i - index_j)^alpha
    return (1 / dist)
end

cutoff = 1e-7
# initialise Hamiltonian
Hamiltonian = zeros(Float32, Dim, Dim)

for Ket = 0:Dim-1
    Diagonal = 0.0

    for i = 0:N-1 #we're doing i!=j so here looping first for i<j (effectively half a sum). Then we multiply by 2 every Hamiltonian contribution to account for all pairs. 
        for j = i+1:N-1

            var_ij = r_distance(i, j, alpha)

            #if cutoff not wanted, comment the following lines

            if var_ij < cutoff
                continue
            end

            #S^Z part, similar to before 
            Spin1 = 2 * ((Ket >> i) & 1) - 1
            Spin2 = 2 * ((Ket >> j) & 1) - 1
            Diagonal += -2.0 * r_distance(i, j, alpha) * J_Z * 0.25 * Spin1 * Spin2


            #S^X part 
            bit_i = 2^i
            bit_j = 2^j
            Bra = Ket ⊻ bit_i ⊻ bit_j
            Hamiltonian[Bra+1, Ket+1] += -2.0 * J_X * 0.25 * r_distance(i, j, alpha)

            si = (Ket >> i) & 1
            sj = (Ket >> j) & 1

            #S^Y part- not all terms have the same sign from the fact that S^Y=1/2i (S^+-S^-). I expanded S^Y*S^Y to check which terms would have an overall minus sign. 

            Bra = Ket ⊻ (bit_i) ⊻ (bit_j)
            sign = (si == sj) ? 1 : -1
            Hamiltonian[Bra+1, Ket+1] += -(-0.25) * 2 * J_Y * r_distance(i, j, alpha) * sign


        end

    end

    Hamiltonian[Ket+1, Ket+1] = Diagonal
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1, Ket+1] = -0.5 * B
    end

end




In [ ]:
using LinearAlgebra
Diag = eigen(Hamiltonian)
GroundState = Diag.vectors[:, 1]  #this gives the groundstate eigenvector
energy = Diag.values
println(length(GroundState))
println("Groundstate eigenvector:$(GroundState)")
println("Corresponding eigenvalue: $(energy[1])")

4096
Groundstate eigenvector:Float32[-0.3309999, -7.9735145f-8, -4.669355f-8, -0.17029724, -6.4539215f-8, -0.04261969, -0.08267266, -1.09422004f-7, -7.108073f-8, -0.034318384, -0.02707013, -4.6589626f-8, -0.10807571, -3.35396f-8, -5.0582187f-8, -0.1371839, -6.807185f-8, -0.017076006, -0.013463883, -4.5167944f-8, -0.02867708, -1.4439111f-8, -2.166962f-8, -0.036381513, -0.08758368, -3.549119f-8, -2.0611257f-8, -0.045107137, -6.491079f-8, -0.021182232, -0.041088153, -1.11953156f-7, -8.922545f-8, -0.01606945, -0.009790162, -4.824401f-8, -0.020845806, -1.5598046f-8, -2.846055f-8, -0.034239136, -0.026871197, -1.40530965f-8, -1.03202f-8, -0.01639085, -3.2242006f-8, -0.0076952167, -0.012604872, -4.3700286f-8, -0.10125933, -3.3012785f-8, -2.441057f-8, -0.06171891, -3.4313643f-8, -0.015457264, -0.025315473, -4.532735f-8, -1.00287885f-7, -0.03228583, -0.019669646, -5.5110835f-8, -0.07857331, -3.8786766f-8, -6.938583f-8, -0.1292025, -7.2534334f-8, -0.009836278, -0.0059909634, -3.9204615f-8, -0.010

In [3]:
magnetizationX = 0.0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    for Bra = 0:Dim-1  #The "easy" (slow) way to do this 
        SumSx = 0.0
        for SpinIndex = 0:N-1  #Loop over spin index 
            bit = 2^SpinIndex   #The bit to be flipped by S+ or S-
            Bra2 = Ket ⊻ bit    #Binary XOR flips the bit
            if Bra == Bra2
                SumSx += 0.5 #the factor of 1/2 above
            end
        end
        magnetizationX += SumSx * GroundState[Ket+1] * GroundState[Bra+1]
    end #Bra
end #Ket
println(magnetizationX / N)

1.080010127424799e-6
